## adjust countries to be the same information

In [104]:
%pip install geopy translate joblib scikit-learn

                                              0.0/8.3 MB ? eta -:--:--
                                              0.1/8.3 MB 2.3 MB/s eta 0:00:04
     -                                        0.2/8.3 MB 3.0 MB/s eta 0:00:03
     --                                       0.5/8.3 MB 4.2 MB/s eta 0:00:02
     ----                                     0.9/8.3 MB 5.4 MB/s eta 0:00:02
     ------                                   1.4/8.3 MB 6.5 MB/s eta 0:00:02
     ---------                                2.0/8.3 MB 7.3 MB/s eta 0:00:01
     -----------                              2.4/8.3 MB 7.6 MB/s eta 0:00:01
     -------------                            2.9/8.3 MB 7.9 MB/s eta 0:00:01
     ---------------                          3.2/8.3 MB 7.9 MB/s eta 0:00:01
     -----------------                        3.6/8.3 MB 7.9 MB/s eta 0:00:01
     -------------------                      4.0/8.3 MB 8.0 MB/s eta 0:00:01
     ---------------------                    4.5/8.3 MB 8.3 MB/s eta 0

In [105]:
%pip freeze > ..//requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Reading files automatically

In [2]:
import os
import pandas as pd

def _adjust_name(name):
    if not('df_' in name):
        return 'df_' + name
    return name
def _get_files_from(base_path = '..\\data\\processed', extension='csv'):
    return [(_adjust_name(name.split('.')[0].lower()), str(os.path.join(root, name))) for (root, _, files) in os.walk(base_path, topdown=False) for name in files if (extension in name)]

def read_files(_from='..\\data\\processed', extension='csv'):
    vars = []
    for item in _get_files_from(_from, extension=extension):
        exec(f'vars.append("{item[0]}")')
        exec(f"{item[0]} = pd.read_csv(r'{item[1]}', sep=';', decimal=',')")
    return vars

# _get_files_from()

In [3]:
df_noaa_global = pd.read_csv(r'..\\data\\processed\\noaa_global\\noaa_global.csv', sep=';', decimal=',')
df_vinhos = pd.read_csv(r'..\\data\\processed\\tech_challenge\\df_vinhos.csv', sep=';', decimal=',')
df_temperature_change_data = pd.read_csv(r'..\\data\\processed\\temp_change\\temperature_change_Data.csv', sep=';', decimal=',')
df_wbpy = pd.read_csv(r'..\\data\\processed\\wbpy\\wbpy.csv', sep=';', decimal=',')

In [4]:
df_noaa_global.head(2)

,station_id,year,stat,value_min,value_mean,value_median,value_max,latitude,longitude,elevation,name
0,ITE00100554,1763,TMAX,-39,147.873973,147.0,309,45.4717,9.1892,150.0,MILAN
1,ITE00100554,1763,TMIN,-63,100.657534,99.0,233,45.4717,9.1892,150.0,MILAN


In [5]:
df_vinhos.head(2)

,pais,ano,quantidade_exportada_pais,valor_exportado_pais,id_produto,produto,quantidade_prod_rs,quantidade_com_rs,country_code
0,Afeganistão,1970,0,0,vinho de mesa,vinho de mesa,217208604.0,98327606.0,af
1,Afeganistão,1970,0,0,vm_tinto,tinto,174224052.0,83300735.0,af


### Adjust df_vinhos pais to country code

### Adjust noaa name to country code

In [6]:
import joblib

base_path = '..\\data\\processed'
dict_latlong = joblib.load(base_path+"\\dict_latlong.pkl")

In [7]:
list_latlong =[f"{_tuple.latitude},{_tuple.longitude}" for _tuple in df_noaa_global[['latitude','longitude']].drop_duplicates().itertuples()]
len(list_latlong)

112035

In [8]:
list_dict = list(dict_latlong.keys())
len(list_dict)

111870

In [9]:
list_diferenca = (list(set(list_latlong) -set(list_dict)))
list_diferenca.sort()
len(list_diferenca)

165

## Load data and adjust temperature

In [2]:
import os
import pandas as pd
import joblib
import numpy as np

base_path = '..\\data\\processed'
dict_latlong = joblib.load(base_path+"\\dict_latlong.pkl")
dict_translates = joblib.load(base_path+"\\dict_translates.pkl")

df_noaa_global = pd.read_csv(r'..\\data\\processed\\noaa_global\\noaa_global.csv', sep=';', decimal=',')
df_vinhos = pd.read_csv(r'..\\data\\processed\\tech_challenge\\df_vinhos.csv', sep=';', decimal=',')
df_temperature_change_data = pd.read_csv(r'..\\data\\processed\\temp_change\\temperature_change_Data.csv', sep=';', decimal=',')
df_wbpy = pd.read_csv(r'..\\data\\processed\\wbpy\\wbpy.csv', sep=';', decimal=',')

In [3]:
df_vinhos.ano.min()

1970

In [4]:
df_vinhos['country_code'] = [dict_translates[x].upper() for x in df_vinhos.pais]

df_noaa_global['lat_long'] = [f"{x.latitude},{x.longitude}" for x in df_noaa_global.itertuples()]
df_noaa_global = df_noaa_global.loc[df_noaa_global['lat_long'].isin(list(dict_latlong.keys()))]
df_noaa_global = df_noaa_global.loc[df_noaa_global['stat'].isin(['TMAX','TMIN','TAVG','PRCP'])]
df_noaa_global = df_noaa_global.loc[df_noaa_global['year'] > 1970]
df_noaa_global['country_code'] = [dict_latlong[f"{x.latitude},{x.longitude}"].upper() for x in df_noaa_global.itertuples()]
df_noaa_global = df_noaa_global.loc[df_noaa_global['country_code'].isin(list(df_vinhos.country_code.unique()))]

In [5]:
def _adjust_temperature(row, var):
    """
    vamos transformar dados em farenheidt para celsius;
    caso seja menor que -90 (menor temperatura ja encontrada no planeta, na antartida) 
    ou maior que 57 (maior temp, na california (death valley, usa)), retorna np.nan
    """
    if (row['stat'] in ['TMAX','TMIN','TAVG']):
        celsius = (row[var] - 32) * 5/9
        if ((celsius > -90) & (celsius < 57)):
            return celsius
        else:
            return np.nan
    return row[var]

for var in ['value_min', 'value_mean', 'value_median','value_max']:
    df_noaa_global[var] = df_noaa_global.apply(lambda row: _adjust_temperature(row, var), axis=1)

In [6]:
df_noaa_global\
     .query('stat=="TMAX"')\
     [['value_min', 'value_mean','value_median','value_max']]\
     .describe()

,value_min,value_mean,value_median,value_max
count,400801.000000,246278.000000,236659.000000,7165.000000
mean,-22.188861,27.198592,27.409438,19.444289
std,42.800676,25.972776,25.449742,29.511280
min,-89.444444,-89.983416,-89.722222,-89.444444
25%,-59.444444,17.246716,16.111111,-0.555556
50%,-26.666667,33.919015,33.888889,26.111111
75%,13.333333,45.554033,46.111111,43.888889
max,56.666667,56.999389,56.944444,56.666667


In [7]:
# imputar media no lugar do nan
for col in ['value_min', 'value_mean', 'value_median','value_max']:
    df_noaa_global[col] = df_noaa_global[col].fillna(df_noaa_global.groupby(['year','stat','country_code'])[col].transform('mean'))

In [8]:
df_noaa_global\
     .query('stat=="TMAX"')\
     [['value_min', 'value_mean','value_median','value_max']]\
     .describe()

,value_min,value_mean,value_median,value_max
count,744802.000000,728837.000000,727992.000000,607930.000000
mean,-22.011482,34.059356,33.512109,23.649294
std,39.544978,16.587638,15.902267,14.556917
min,-89.444444,-89.983416,-89.722222,-89.444444
25%,-48.888889,34.356388,32.903462,14.763889
50%,-35.519751,37.020081,36.022710,25.134100
75%,19.444444,41.676949,40.555556,32.425214
max,56.666667,56.999389,56.944444,56.666667


In [9]:
df_noaa_global\
    .groupby(['year','stat','country_code'])\
    .agg({
        'value_min':('min','mean','median','max')
    })\
    .groupby(['stat'])\
    .describe()

value_min                                                              
           min                                                              
         count       mean         std           min        25%        50%   
stat                                                                        
PRCP    4549.0 -13.956474  612.847574 -30866.000000   0.000000   0.000000  \
TAVG    2772.0 -34.255451   46.531585    -89.444444 -79.444444 -40.833333   
TMAX    2659.0 -36.241486   44.226140    -89.444444 -76.666667 -45.555556   
TMIN    3733.0 -34.701164   50.644788    -89.444444 -83.333333 -52.222222   

                                                 ...                           
                                mean             ...     median                
            75%          max   count       mean  ...        75%          max   
stat                                             ...                           
PRCP   0.000000  4361.000000  4549.0   4.465069  ...   0.000000  4361.000000  \
TAVG   6.111111    56.666667  2772.0  -9.206290  ...  27.777778    56.666667   
TMAX  -5.555556    56.666667  2659.0  -7.688848  ...  31.666667    56.666667   
TMIN  19.444444    56.666667  3733.0 -10.972411  ...  32.222222    56.666667   

                                                                       
         max                                                           
       count       mean         std        min        25%        50%   
stat                                                                   
PRCP  4549.0  45.032095  331.024009   0.000000   0.000000   0.000000  \
TAVG  2772.0  14.720819   42.687173 -88.888889 -20.555556  31.111111   
TMAX  2659.0  18.540596   39.554338 -87.777778 -17.777778  37.777778   
TMIN  3733.0  15.800667   41.762977 -89.444444 -16.111111  34.444444   

                              
                              
            75%          max  
stat                          
PRCP   0.000000  9271.000000  
TAVG  52.222222    56.666667  
TMAX  54.444444    56.666667  
TMIN  51.666667    56.666667  

[4 rows x 32 columns]

In [10]:
df_noaa_global\
    .groupby(['year','stat','country_code'])\
    .agg({
        'value_mean':('mean')
    })\
    .groupby(['stat'])\
    .describe()

value_mean                                                           
          count       mean         std        min        25%        50%   
stat                                                                      
PRCP     4549.0  56.040772  133.335521   0.000000  20.304621  32.565068  \
TAVG     1955.0  29.295964   19.049174 -55.484018  18.608652  35.391439   
TMAX     1638.0  33.804528   16.658031 -29.777778  24.180306  36.793634   
TMIN     2648.0  18.924296   21.171263 -54.512463   4.743791  22.163609   

                              
            75%          max  
stat                          
PRCP  59.945622  4361.000000  
TAVG  42.692467    56.889645  
TMAX  47.286344    56.879756  
TMIN  33.873764    56.937248

In [11]:
df_noaa_global\
     .query('country_code=="BR"')\
     .groupby(['stat','country_code'])\
     [['value_min', 'value_mean','value_max']]\
     .describe()

value_min                                               
                      count       mean        std        min        25%   
stat country_code                                                         
PRCP BR             96827.0   0.326613  29.725595   0.000000   0.000000  \
TAVG BR              3128.0  37.224406   8.012492  -6.111111  34.343434   
TMAX BR              1543.0  48.004916   5.129841  32.222222  44.920635   
TMIN BR              2472.0  26.708150  14.737536 -56.666667  21.150794   

                                                     value_mean              
                         50%        75%          max      count       mean   
stat country_code                                                            
PRCP BR             0.000000   0.000000  8880.000000    96827.0  38.969557  \
TAVG BR            37.083333  41.923077    56.666667      449.0  52.551039   
TMAX BR            48.333333  51.944444    56.666667        0.0        NaN   
TMIN BR            26.240741  35.804094    56.666667     2220.0  42.659277   

                   ...                         value_max               
                   ...        75%          max     count        mean   
stat country_code  ...                                                 
PRCP BR            ...  46.789730  8880.000000   96827.0  824.395551  \
TAVG BR            ...  54.861111    56.666667     262.0   51.921120   
TMAX BR            ...        NaN          NaN       0.0         NaN   
TMIN BR            ...  48.888889    56.944444    1975.0   38.636021   

                                                                               
                          std        min         25%         50%         75%   
stat country_code                                                              
PRCP BR            362.748694   0.000000  620.000000  780.000000  968.000000  \
TAVG BR              3.721388  47.777778   47.777778   51.111111   56.666667   
TMAX BR                   NaN        NaN         NaN         NaN         NaN   
TMIN BR             12.628032 -17.777778   32.222222   40.416667   48.888889   

                                
                           max  
stat country_code               
PRCP BR            9990.000000  
TAVG BR              56.666667  
TMAX BR                    NaN  
TMIN BR              56.666667  

[4 rows x 24 columns]

In [12]:
dest_path = '..\\data\\processed\\noaa_global\\'
df_noaa2 = df_noaa_global\
    .groupby(['year','stat','country_code'])\
    .agg({
        'value_min':'min'
        , 'value_mean':'mean'
        , 'value_median':'median'
        , 'value_max':'max'
    })\
    .reset_index()
df_noaa2\
    .to_csv(dest_path+'noaa_global_final.csv', index=False, sep=';', decimal=',')

In [80]:
dest_path = '..\\data\\processed\\tech_challenge\\'
df_vinhos.to_csv(dest_path+'df_vinhos.csv', index=False, sep=';', decimal=',')
